In [98]:
import cv2
import numpy as np
fourmis = []
id_counter = 0
velocities = []

v = cv2.VideoCapture('video_boite_entiere-test.ts')
bg = cv2.imread("bg.png")
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)
bg = cv2.GaussianBlur(bg, (21, 21), 0)

fps = v.get(cv2.CAP_PROP_FPS)
duration = 10  # durée en secondes
frames_to_capture = int(fps * duration)

for i in range(frames_to_capture):
    
    ok, frame = v.read()

    if not ok:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # Calculer la différence entre le background et la frame actuelle
    diff = cv2.absdiff(bg, gray)
    thresh = cv2.threshold(diff, 35, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Trouver les contours et les boîtes englobantes
    cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    nb= 0
    for c in cnts:
        # Calculer la boîte englobante
        (x, y, w, h) = cv2.boundingRect(c)
        center = (int(x + w/2), int(y + h/2))
        #(x,y,w,h) = cv.boundingRect(c) # coord
        cv2.putText(frame,"fourmis",(x,y),cv.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)

        # Calculer la vitesse
        if len(fourmis) >= 2:
            (x2, y2, w2, h2) = fourmis[-2]['rect']
            center2 = (int(x2 + w2/2), int(y2 + h2/2))
            dx, dy = np.subtract(center, center2)
            dt = 1/v.get(cv2.CAP_PROP_FPS)
            velocities.append((dx/dt, dy/dt))

        # Identifier la fourmi
        id = None
        if len(fourmis) == 0:
            id = id_counter
            id_counter += 1
        else:
            distances = [np.linalg.norm(np.array(center) - f['center']) for f in fourmis]
            if min(distances) < 50 :
                nb+=1
                id = distances.index(min(distances))
            else:
                id = id_counter
                id_counter += 1

        # Ajouter la fourmi à la liste
        fourmis.append({
            'id': id,
            'rect': (x, y, w, h),
            'center': center,
            'velocity': velocities[-1] if len(velocities) > 0 else (0, 0)
        })
v.release()
cv2.destroyWindows()

AttributeError: module 'cv2' has no attribute 'destroyWindows'

In [99]:
print(fps)

25.0


In [90]:
res = []
for f in fourmis : 
    res.append(f['id'])
id_fourmis,count = np.unique(res,return_counts=True)

In [93]:
print(nb)

10


In [92]:
print(len(id_fourmis))

1352


In [82]:
import pandas as pd
DataFrame = pd.DataFrame(fourmis)
print(DataFrame)

        id                 rect       center             velocity
0        0    (1002, 232, 5, 5)  (1004, 234)               (0, 0)
1        0    (1002, 232, 6, 7)  (1005, 235)               (0, 0)
2        1    (491, 1040, 7, 6)  (494, 1043)  (-12750.0, 20225.0)
3        1    (1002, 231, 8, 8)  (1006, 235)          (25.0, 0.0)
4        2    (490, 1039, 7, 7)  (493, 1042)       (-25.0, -25.0)
...    ...                  ...          ...                  ...
3788  3778   (543, 753, 15, 11)   (550, 758)   (-6000.0, -2825.0)
3789  1007   (526, 628, 10, 13)   (531, 634)   (-7650.0, -4325.0)
3790  3602   (490, 335, 17, 19)   (498, 344)  (-1300.0, -10350.0)
3791  3781  (1030, 310, 17, 17)  (1038, 318)   (12675.0, -7900.0)
3792  3772  (1016, 155, 13, 15)  (1022, 162)   (13100.0, -4550.0)

[3793 rows x 4 columns]
